### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{},{},1d6a5fdi
1,{},{},3oeflkwx
2,{},{},g7brabgk
3,{},{},3euqgs3f
4,{},{},1oyem9jr
...,...,...,...
763,"{'_wandb': {'runtime': 20220}, '_runtime': 201...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
764,"{'_step': 117001, '_wandb': {'runtime': 18252}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
765,"{'_runtime': 18375, '_timestamp': 1659280529, ...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
766,"{'_timestamp': 1659279862, 'negative_sim': 0.1...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline_10'
exp_name = 'curl'
model_path = '0/100/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/home/nas3_userK/hojoonlee/anaconda3/envs/atari/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,summary,config,id,env
8,"{'pos_sim': 0.9698366914177896, 'reward_f1': 0...","{'env': {'game': 'UpNDown', 'type': 'atari', '...",ifktwftq,UpNDown
9,"{'pos_sim': 0.9535219297373204, '_runtime': 74...","{'env': {'game': 'Seaquest', 'type': 'atari', ...",2wbtk30m,Seaquest
10,"{'_step': 234400, 'reward_ratio': 0.0746537326...","{'env': {'game': 'RoadRunner', 'type': 'atari'...",2kwjwit8,RoadRunner
11,"{'_runtime': 73176, 'reward_ratio': 0.00455022...","{'env': {'game': 'PrivateEye', 'type': 'atari'...",3cyuy2ls,PrivateEye
12,"{'loss': 0.1533650455431532, 'neg_sim': -0.000...","{'env': {'game': 'Qbert', 'type': 'atari', 'fr...",3t53bt1q,Qbert
13,"{'reward_f1': 0.9081885856079404, 'pos_neg_dif...","{'env': {'game': 'Pong', 'type': 'atari', 'fra...",trcpnw2z,Pong
21,"{'max_grad_norm': 0.27114126754733253, 'reward...","{'env': {'game': 'MsPacman', 'type': 'atari', ...",1jcez040,MsPacman
22,"{'best_metric_val': 0.24958297984699004, '_wan...","{'env': {'game': 'KungFuMaster', 'type': 'atar...",rbc3nn1m,KungFuMaster
23,"{'_timestamp': 1665958379, 'epoch': 100, 'act_...","{'env': {'game': 'Kangaroo', 'type': 'atari', ...",2do11c18,Kangaroo
24,"{'loss': 0.09998832872711626, '_wandb': {'runt...","{'env': {'game': 'Krull', 'type': 'atari', 'fr...",2ys5byoe,Krull


### Restore Saved Models

In [6]:
import pathlib
base_path = str(pathlib.Path().resolve())

artifact_dict = {}
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    name = env + '/' + model_path 
    path = base_path + '/' + name
    wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
    
    artifact_dict[path] = name 

UpNDown ifktwftq
Seaquest 2wbtk30m
RoadRunner 2kwjwit8
PrivateEye 3cyuy2ls
Qbert 3t53bt1q
Pong trcpnw2z
MsPacman 1jcez040
KungFuMaster rbc3nn1m
Kangaroo 2do11c18
Krull 2ys5byoe
Jamesbond 9bwnzfrw
Hero 13m14gnx
Frostbite 29ue1rxx
Gopher 3ikhbaq1
Freeway 1ejbwhzb
ChopperCommand kdr8j33h
CrazyClimber 9qpbaxlz
DemonAttack 1rko9q8v
Breakout 333g9s63
Boxing 3n2yspti
BankHeist 1nfz1pcf
BattleZone 2fa3bfh1
Asterix 8mdu2twg
Assault 1oyr64u5
Amidar 1cg7ksxq
Alien 36kn4snd


### Save to artifact

In [7]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


### Remove artifact

In [8]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### Generate json file

In [17]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)